### RNN

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from tensorflow.python.keras.layers import Dense, Activation, Dropout, SimpleRNN
from tensorflow.python.keras.models import Sequential, load_model

In [15]:
# load data
import pandas as pd
df = pd.read_csv('data/bike_rnn.csv', sep=',', header=None)
print(df.values)

[['num' 'weekday' 'hour']
 ['0' '5' '17']
 ['1' '5' '17']
 ...
 ['6' '2' '15']
 ['6' '2' '15']
 ['6' '2' '15']]


In [16]:
bikes = []
def data_bike_num(path_to_dataset='data/bike_rnn.csv',
                           sequence_length=20,
                           ratio=1.0):
    max_values = ratio * 45949
    with open(path_to_dataset) as f:
        data = csv.reader(f, delimiter=",")
        next(data, None)  # skip the headers
        nb_of_values = 0
        for line in data:
            try:
                bikes.append(float(line[0]))
                nb_of_values += 1
            except ValueError:
                pass
            if nb_of_values >= max_values:
                break
    print ("Data loaded from csv. Formatting...")
    print(len(bikes))

    result = []
    for index in range(len(bikes) - sequence_length):
        result.append(bikes[index: index + sequence_length])
    result = np.array(result)  # shape (2049230, 50)
    result_mean = result.mean()
    result -= result_mean
    print("Shift: ", result_mean)
    print ("Data: ", result.shape)
    row = int(round(0.95 * result.shape[0]))
    train = result[:row, :]
    np.random.shuffle(train)
    X_train = train[:, :-1]
    y_train = train[:, -1]
    X_test = result[row:, :-1] # 2297
    y_test = result[row:, -1]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    return [X_train, y_train, X_test, y_test, result_mean]


In [ ]:
# Function to build the RNN-based model
def build_model():
    model = Sequential()
    layers = [1, 50, 100, 1]
    model.add(SimpleRNN(
        layers[1],
        input_shape=(None, layers[0]),
        return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(layers[3]))
    model.add(Activation("linear"))
    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop", metrics=['mae', 'mape'])
    print ("Compilation Time : ", time.time() - start)
    return model

# We are now ready to train the model we defined on our data.
global_start_time = time.time()
epochs = 2
ratio = 1
sequence_length = 20
path_to_dataset = 'data/bike_rnn.csv'
X_train, y_train, X_test, y_test, result_mean = data_bike_num(
    path_to_dataset, sequence_length, ratio)

print ('\nData Loaded. Compiling...\n')
model = build_model()
model.fit(
    X_train, y_train,
    batch_size=512, epochs=epochs, validation_split=0.05)
predicted = model.predict(X_test)
predicted = np.reshape(predicted, (predicted.size,))


Data loaded from csv. Formatting...
45949
Shift:  6.997708419517081
Data:  (45929, 20)

Data Loaded. Compiling...

Compilation Time :  0.005113840103149414


AttributeError: module 'tensorflow.python.distribute.input_lib' has no attribute 'DistributedDatasetInterface'